<div style="text-align: left; background-color:#FFFFCC; font-family:Arial; color:#526085; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 58px;border-style: solid;border-color: dark green;">Lab 02: Working with time-series data</div>
<div style="text-align: left; background-color:#99FFFF; font-family: Arial; color:#551184; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 18px">Bộ môn: Trực quan hóa dữ liệu | Nhóm 8</div>

# <div style="text-align: left; background-color:#0C6A86; font-family:Arial; color:white; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 30px;border-style: solid;border-color: black;">PHẦN 1. Tiền xử lý & khám phá dữ liệu</div>

# Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go

# Nội dung chính

## Đọc dữ liệu

In [2]:
df = pd.read_csv("Data/data_18_19_processed.csv")
df.head()

,Reference Number,Grid Ref: Easting,Grid Ref: Northing,Number of Vehicles,1st Road Class,1st Road Class & No,Road Surface,Lighting Conditions,Weather Conditions,Local Authority,Type of Vehicle,Casualty Class,Casualty Severity,Sex of Casualty,Age of Casualty,Full Time
0,51B0230,433936,428874,1,6,U,1,4,1,E08000035,9,3,3,2,87,2018-01-11 07:00:00
1,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,60,2018-01-11 08:55:00
2,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,51,2018-01-11 08:55:00
3,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,50,2018-01-11 08:55:00
4,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,51,2018-01-11 08:55:00


### Biểu đồ

##### **Tiêu đề**: thống kê số các tai nạn theo nhóm tuổi và giới tính

##### **Loại biểu đồ**: Biểu đồ hình tháp (Pyramid plot)

#### **Lý do chọn**: 
- **Trực quan hóa dễ hiểu**: Biểu đồ tháp là một công cụ rất thích hợp để biểu diễn và so sánh theo độ tuổi giữa 2 nhóm cá thể một cách trực quan nhất, giúp người nhìn có thể nhanh chóng nắm bắt thông tin.
- **Dễ dàng xác định được kết quả cần so sánh**: dựa vào đồ thị có thể dễ dàng thấy được sự chênh lệch khi thực hiện so sánh một giá trị nào đó liên quan đến nội dung đồ thị

##### Trực quan hóa

In [3]:
df['Sex of Casualty'] = df['Sex of Casualty'].replace({1: 'Male', 2: 'Female'})
bins = [1, 16, 30, 50, 65, 80, 95]
labels = ['1-16','16-30', '30-50', '50-65','65-80','80-95']
df['Age Group'] = pd.cut(df['Age of Casualty'], bins=bins, labels=labels, right=False)
agesex_df = df.pivot_table(index='Sex of Casualty', columns='Age Group', aggfunc='size', fill_value=0)
age_groups = agesex_df.columns
male_counts = agesex_df.loc['Male']
female_counts = agesex_df.loc['Female'] * -1  

fig = go.Figure()
fig.add_trace(go.Bar(
    y=age_groups,
    x=male_counts,
    name='Male',
    orientation='h',
    marker=dict(color='#800000')
))
fig.add_trace(go.Bar(
    y=age_groups,
    x=female_counts,
    name='Female',
    orientation='h',
    marker=dict(color='purple')
))
fig.update_layout(
    title='Population Pyramid by Age Group and Sex',
    barmode='overlay',
    bargap=0.1,
    xaxis=dict(
        title='Number of people',
        tickvals=[-max(abs(female_counts)), 0, max(male_counts)],
        ticktext=[max(abs(female_counts)), 0, max(male_counts)],
    ),
    yaxis=dict(title='Age Group'),
    template='plotly_white'
)
fig.show()

C:\Users\Lyliths\AppData\Local\Temp\ipykernel_7928\1604365046.py:5: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  agesex_df = df.pivot_table(index='Sex of Casualty', columns='Age Group', aggfunc='size', fill_value=0)


#### **Kỹ thuật trực quan được sử dụng**:
- **Màu sắc**: Sử dụng 2 màu khác nhau để biểu diễn cho đồ thị, giúp cho người xem có thể phân biệt nhanh chóng giữa 2 cá thể được đề cập trong đồ thị, ngoài ra còn giúp đồ thị rõ ràng hơn
- **Kỹ thuật Manipulate View**:
    - **Hiển thị thông tin qua tooltip**: Khi đưa con trỏ chuột vào một thành phần của đồ thị, nó sẽ hiện lên số lượng cá thể và nhóm tuổi của cá thể đó

⇒ Đồ thị trực quan, rõ ràng, dễ nắm bắt thông tin cần thiết một cách nhanh chóng và dễ tương tác với người dùng

### Nhận xét: 
- Dựa vào đồ thị, ta dễ dàng thấy được số lần gặp tai nạn trải dài ở các độ tuổi từ 1 đến 95 tuổi. 
- Đa số nam giới sẽ gặp tai nạn nhiều hơn so với nữ giới. Ở độ tuổi 30-50, số lượng nam giới gặp tai nạn gần gấp đôi so với nữ giới, tương tự với nhóm tuổi 16-30 tuổi.
- Độ tuổi có số lần tai nạn ít nhất là 80-95 tuổi ở cả nam và nữ, độ tuổi có số lượng tai nạn nhiều nhất là 30-50 tuổi, kế đến là từ 16-30 tuổi.

### Biểu đồ

#### **Tiêu đề**: Mức độ nghiêm trọng của tai nạn và mức độ ảnh hưởng đến các phương tiện khác

#### **Loại biểu đồ**: Biểu đồ đa cột(Grouped bar plot)

#### **Lý do chọn**: 
- **Trực quan hóa dễ hiểu**: biểu đồ đa cột là dạng biểu đồ thích hợp nhất biểu diễn hiệu quả nhất loại dữ liệu này một cách trực quan và rõ ràng nhất
- **Dễ dàng xác định được kết quả cần so sánh**: có thể dễ dàng thấy được sự chênh lệch khi thực hiện so sánh một giá trị nào đó liên quan đến nội dung đồ thị, cũng như có thể thấy được xu hướng của đồ thị tập trung vào loại giá trị nào

#### Trực quan hóa

In [4]:
impact_df = df.pivot_table(index='Casualty Severity', columns='Number of Vehicles', aggfunc='size', fill_value=0)
impact_df = impact_df.reset_index()
number_of_vehicles = impact_df.columns[1:]

fig = go.Figure()

for vehicle in number_of_vehicles:
    fig.add_trace(go.Bar(
        x=impact_df['Casualty Severity'],
        y=impact_df[vehicle],
        name=f'Number of Vehicles: {vehicle}'
    ))
fig.update_layout(
    title='Number of vehicles relative to the Casualty Severity',
    xaxis=dict(title='Casualty Severity'),
    yaxis=dict(title='Number of Accidents'),
    barmode='group',
    width=1000,
    height=600
)

fig.show()

#### **Kỹ thuật trực quan được sử dụng**:
- **Màu sắc**: Sử dụng các màu khác nhau để biểu diễn cho số lượng các phương tiện bị ảnh hưởng, giúp cho người xem có thể phân biệt nhanh chóng các cá thể được đề cập trong đồ thị, ngoài ra còn giúp đồ thị rõ ràng hơn
- **Kỹ thuật Manipulate View**:
    - **Hiển thị thông tin qua tooltip**: Khi người dùng trỏ chuột vào từng cột trong biểu đồ, một tooltip sẽ xuất hiện hiển thị số lượng sự kiện cụ thể tại mức độ thương tích đó. Điều này giúp người dùng có thể xem thông tin chi tiết mà không cần phải xem toàn bộ dữ liệu cùng một lúc.
    - **Tùy chỉnh giao diện hiển thị**: Biểu đồ cột có thể được thiết kế để cho phép người dùng có thể chọn các cột hiển thị để quan sát các đối tượng người dùng quan tâm

⇒ Biểu đồ trực quan, rõ ràng, giúp người dùng có thể dễ nắm bắt thông tin một cách nhanh chóng và hiệu quả nhất

### Nhận xét: 
- Từ đồ thị, ta có thể thấy được số tai nạn có mức độ nghiêm trọng loại 3 chiếm số lượng áp đảo.
- Trong số đó, khi có tai nạn loại 3 xảy ra, thường sẽ có 3 phương tiện khác bị ảnh hưởng. con số này gần gấp ba lần so với giá trị thấp sau nó.
- Trong đa số các mức độ nghiêm trọng, thường thì 3 phương tiện bị ảnh hưởng chiếm số lượng lớn nhất, tuy nhiên đối với tai nạn loại 1, 1 phương tiện bị ảnh hưởng có nhiều hơn so với 3 phương tiện ảnh hưởng như 2 mức độ nghiêm trọng khác
- Đối với trường hợp có từ 4 phương tiện bị ảnh hưởng bởi tai nạn, may mắn thay có rất ít trường hợp như thế, điều đó chứng tỏ số lượng tai nạn gây ảnh hưởng xung quanh vẫn còn ít.

### Biểu đồ

#### **Tiêu đề**: Dự đoán mức nghiêm trọng của tai nạn bằng mô hình k-NN

#### **Loại biểu đồ**: Biểu đồ phân tán (Scatter Plot)

#### **Lý do chọn**: 
- Biểu đồ phân tán rất thích hợp để xem xét mức độ chính xác giữa giá trị dự đoán và giá trị thực tế.
- Với sự trợ giúp của plotly, ta dễ dàng so sánh kết quả thu được sau khi thực hiện áp dụng model vào dữ liệu

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [6]:
selected_columns = ['Road Surface', 'Lighting Conditions', 'Weather Conditions', 'Casualty Severity']
subset_df = df[selected_columns]
X = subset_df.drop('Casualty Severity', axis=1)
y = subset_df['Casualty Severity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

result_df_rf = pd.DataFrame({
    'Actual Data': y_test,
    'Predicted Data': y_pred
})

trace_actual = go.Scatter(
    x=result_df_rf.index, 
    y=result_df_rf['Actual Data'], 
    mode='markers', 
    name='Actual Data', 
    marker=dict(color='blue')
)
trace_predicted = go.Scatter(
    x=result_df_rf.index, 
    y=result_df_rf['Predicted Data'], 
    mode='markers', 
    name='Predicted Data', 
    marker=dict(color='red')
)

layout_rf = go.Layout(
    title='Predict Value compare to Actual Value',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Casualty Severity'),
    hovermode='closest'
)

fig_rf = go.Figure(data=[trace_actual, trace_predicted], layout=layout_rf)
fig_rf.show()

#### **Kỹ thuật trực quan được sử dụng**:
- **Màu sắc**: Sử dụng 2 màu khác nhau để biểu diễn cho dữ liệu dự đoán và dữ liệu thực tế, giúp phân biệt rõ ràng
- **Kỹ thuật Manipulate View**:
    - **Hiển thị thông tin qua tooltip**: Khi người dùng trỏ chuột vào từng điểm trong biểu đồ, sẽ có 1 bảng thông tin hiện ra cho ta biết thứ tự và giá trị của điểm đó
    - **Tùy chỉnh giao diện hiển thị**: Biểu đồ phân tán có thể được thiết kế để cho phép người dùng có thể chọn hiển thị giá trị thực tế hoặc giá trị dự đoán để dễ quan sát hơn

⇒ Biểu đồ trực quan, giúp cho người dùng có thể tiếp cận nhanh chóng sự khác biệt giữa giá trị dự đoán và giá trị thực tế

### Nhận xét: 
- Từ đồ thị, ta có thể thấy được model k-NN cho kết quả dự đoán khá chính xác so với giá trị thực tế.
- Đa số các giá trị dự đoán đều tương đối chính xác khi so với dữ liệu thực tế, tuy nhiên vẫn còn sai sót và chưa khớp nhiều.
- Có thể xem model k-NN này là một mô hình phù hợp cho việc dự đoán mức độ nghiêm trọng của tai nạn dựa trên các thông tin được lựa chọn ở trên. Nếu tiếp tục cải tiến model có thể sẽ đạt được độ chính xác tốt hơn.